In [35]:
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig
from shapely.geometry import LineString,shape, mapping
import os, glob
import pandas as pd
import datetime
import numpy as np
from pandas import HDFStore
from tqdm import tqdm
from haversine import haversine
import math

In [ ]:
nodes = pd.read_csv('data/SanFrancisco_nodes.csv')
edges = pd.read_csv('data/SanFrancisco_edges.csv')

In [ ]:
network = Network("small-Sanf/edges.shp","fid", "u", "v")
print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
graph = NetworkGraph(network)

In [ ]:
ubodt = UBODT.read_ubodt_csv("network-SanFrancisco/ubodt.txt")

fmm_model = FastMapMatch(network,graph,ubodt)

k = 8
radius = 0.003
gps_error = 0.0005
fmm_config = FastMapMatchConfig(k,radius,gps_error)

In [ ]:
path = os.getcwd() +'/data/cabspottingdata/Multiple_Trajectories/Single_Trajectory/'
file_list = os.listdir(path)
f_name=[]
for filename in file_list:
    if os.path.splitext(filename)[1] == '.csv':
        f_name.append(os.path.splitext(filename)[0])

In [ ]:
def move1sec(start, destination, speed):

    Lat1 = math.radians(start[0])
    Lat2 = math.radians(destination[0])
    Lng1 = math.radians(start[1])
    Lng2 = math.radians(destination[1])

    y = math.sin(Lng2-Lng1)*math.cos(Lat2)
    x = math.cos(Lat1)*math.sin(Lat2)-math.sin(Lat1)*math.cos(Lat2)*math.cos(Lng2-Lng1)
    z = math.atan2(y, x)

    a = np.rad2deg(z)

    brng = math.radians(a)

    R = 6378.1
    d = (((speed * 5) / 18) / 1000)

    lat2 = math.asin(math.sin(Lat1) * math.cos(d/R) + math.cos(Lat1) * math.sin(d/R) * math.cos(brng))
    lon2 = Lng1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(Lat1),math.cos(d/R)-math.sin(Lat1)*math.sin(Lat2))
    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    
    return lat2, lon2

In [ ]:
a = np.zeros((1,4468))
a = a.astype('int16')
list1 = [[] for i in range(4468)]
start = datetime.datetime(2008, 5, 17, 10, 0, 0)

In [ ]:
for t in tqdm(f_name):
    df = pd.read_csv(path + t + '.csv', index_col = 0)
    
    for wkt, ti, cnt in tqdm(zip(df['LineString'], df['StartTime'], df['Num_p'])):
        result = fmm_model.match_wkt(wkt, fmm_config)
        
        candidates = []
        for c in result.candidates:
            candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))
            
        df_c = pd.DataFrame(candidates, columns=["eid","source","target","error","length","offset","spdist","ep","tp"])
        df_c['speed'] = df_c['spdist'] * 6000
        
        temp_point = result.pgeom.export_wkt()
        now_time = ti
        
        if now_time.count(':') == 1:
            now_time += ':00'
        cnow_time = datetime.datetime.strptime(now_time, "%Y-%m-%d %H:%M:%S")
        cnow_time = cnow_time.replace(second=0)
        cnow_time += datetime.timedelta(minutes=1)
        point = temp_point[11:-1].split(',')
        
        for i in range(1, len(df_c) -1):
            st = str(cnow_time.weekday()) + ':' + str(cnow_time.hour) + ':' + str(cnow_time.minute)
            checkM = 0
            flag = 0
            temp = point[i].split(' ')
            cCodi = (float(temp[1]), float(temp[0]))
            id = nodes.loc[nodes['osmid']== edges['u'][df_c['eid'][i]]].index[0]
            temp = nodes.geometry[id][7:-1].split(' ')
            sCodi = (float(temp[1]), float(temp[0]))
            count = 0
            est = df_c['speed'][i]
            if est <= 5:
                continue
            flag = 1
            while haversine(cCodi, sCodi, unit='m') >= (est * 5) / 18:
                cCodi = move1sec(cCodi, sCodi, est)
                count += 1
                if count >= 60:
                    continue
            if flag == 1:
                while df_c['eid'][i] == df_c['eid'][i + 1]:
                    if i == len(df_c) - 2:
                        break
                    i += 1
                    checkM += 1
                temp = point[i].split(' ')
                cCodi = (float(temp[1]), float(temp[0]))
                p3Time = count + (checkM * 60)
                id = nodes.loc[nodes['osmid']== edges['v'][df_c['eid'][i]]].index[0]
                temp = nodes.geometry[id][7:-1].split(' ')
                dCodi = (float(temp[1]), float(temp[0]))
                count = 0
                est = df_c['speed'][i + 1]
                if est <= 5:
                    continue
                while haversine(cCodi, dCodi, unit='m') >= (est * 5) / 18:
                    cCodi = move1sec(cCodi, dCodi, est)
                    count += 1
                    if count >= 60:
                        continue
                p4Time = p3Time + count
                flag = 0
            
            list1[df_c['eid'][i]].append(st+', ' + str(p4Time))

In [ ]:
idi = pd.DataFrame(list1)
idi.to_csv('small-Sanf/queue.csv')